In [1]:
import os
import sys
import types
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sas

import read_clean_quote as rcq
import calculate_trends as ct

In [2]:
small_list = [3,5,10]
long_list  = [5,10,20,30]

In [3]:
tempDf = rcq.read_quote_data('IBM')

rcq.fill_quote_gaps( tempDf )
rcq.fix_splits( tempDf )

In [5]:
diffs = ct.generate_differentials(   tempDf                    ) # Differentials between c,ac, o, h, l, v
rolls = ct.generate_rolling_close(   tempDf,        small_list ) # Rolling averages of adj c, listed as C_mean_n and C_std_10
moms  = ct.generate_momentum_close(  tempDf,         long_list ) # Momentum between day and days offset in list, Momentum_n
rsis  = ct.generate_rsi(             tempDf, diffs,  long_list ) # Relative Strength Indexes over n days, RSI_n
bolls = ct.generate_bollinger_bands( tempDf, rolls, small_list ) # percentB of Bollinger bands, 1 is at upper, 0 at lower

In [6]:
# Join everything, or can leave as seperate 
tempDf = tempDf.join( diffs, how='inner' )
tempDf = tempDf.join( rolls, how='inner' )
tempDf = tempDf.join(  moms, how='inner' )
tempDf = tempDf.join(  rsis, how='inner' )
tempDf = tempDf.drop( ['Open','High','Low','Volume','Close'], axis=1 )

In [6]:
tempDf.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2017-03-01,180.479996,182.550003,180.029999,181.949997,2960000,181.949997
2017-02-28,179.380005,180.630005,179.350006,179.820007,3272500,179.820007
2017-02-27,181.190002,181.250000,179.279999,179.399994,3672000,179.399994
2017-02-24,180.210007,181.490005,179.889999,181.350006,3267800,181.350006
2017-02-23,182.000000,182.500000,180.919998,181.649994,2250500,181.649994


In [8]:
# Data should be cleaned, need to plot some trends and check for correlation
# [ 'Diff_co', 'Diff_ao', 'Diff_hl', 'Diff_v'] ]
# [ 'Close_mean_n', 'Close_std_n' ]
# [ 'Momentum_n' ]
#
# Diff co, v, have different info, co same as hl basically
# Momentum 3 correlated with co, 5 a little less, 10 at limit
# HUGE autocorrelation with momentum, long momentum really tracks trends, until at/past scale of momentum